Cross Validation 3

size =  72684

test size 4k+

epoch-5

apply sigmoid (prediction probabilities are logits)

groups-9

target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

threshold-tuning = yes

df_2k = df[(df['Rating'] >= 4.0) & (df['Maximum_Installs'] >= 20000)]

accuracy score : F1 score (micro) and ROC_AUC score

## 1. Setup

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
print(tf.__version__)

2.8.0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load Dataset

In [4]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [5]:
## uploading csv files on drive (to avoid uploading on colab in every session)

from google.colab import drive
drive.mount("/content/drive/")

## drive path
train_path = "/content/drive/MyDrive/MetadataCSV/data_72684/CV_df_train_3.csv"
val_path = "/content/drive/MyDrive/MetadataCSV/data_72684/CV_df_val_3.csv"
test_path = "/content/drive/MyDrive/MetadataCSV/test_dataset.csv"

Mounted at /content/drive/


In [8]:
df_train = pd.read_csv(train_path) 
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(58147, 23)
(14537, 23)
(4624, 23)


In [9]:
df_train.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Face Chat,com.facechat.live,Social,4.1,7306795,False,Enjoy chatting? Social and Video Chat on Face ...,https://sites.google.com/view/xender-chat-term...,0,1,...,0,0,1,1,0,0,1090,enjoy chatting? social and video chat on face ...,1025,197
1,Running - Calorie Counter,com.sdgcode.runningcaloriecounter,Health & Fitness,4.1,286133,False,"Running Counter - Calorie Counter app, your wa...",http://app.sdgcode.com/privacy-policy/,0,0,...,0,0,0,0,0,0,845,"running counter calorie counter app, your way ...",816,162


In [10]:
target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

In [11]:
# getting number of nonzeros in each column
df_train[target_list].astype(bool).sum(axis=0)

Camera         9295
Location       9208
Microphone     5463
Contacts       6315
Storage       28498
Phone          8948
SMS             276
Call_Log        182
Calendar       1032
dtype: int64

In [12]:
df_val[target_list].astype(bool).sum(axis=0)

Camera        2242
Location      2314
Microphone    1323
Contacts      1619
Storage       7110
Phone         2288
SMS             76
Call_Log        45
Calendar       263
dtype: int64

In [13]:
df_test[target_list].astype(bool).sum(axis=0)

Camera         745
Location       716
Microphone     436
Contacts       500
Storage       2402
Phone          652
SMS             11
Call_Log         6
Calendar        90
dtype: int64

## 3. Data Preprocess

#### 3.1 Clean Text

In [14]:
import nltk

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [16]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(remove_stopwords)
df_val["Clean_Description"] = df_val["Clean_Description"].map(remove_stopwords)
df_test["Clean_Description"] = df_test["Clean_Description"].map(remove_stopwords)

In [17]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [18]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(lambda x: remove_punct(x))
df_val["Clean_Description"] = df_val["Clean_Description"].map(lambda x: remove_punct(x))
df_test["Clean_Description"] = df_test["Clean_Description"].map(lambda x: remove_punct(x))

#### 3.2 Create Corpus

In [19]:
from keras.layers import *

In [20]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [21]:
from nltk.tokenize import word_tokenize
import nltk


def create_corpus_tk(df):
    corpus = []
    for text in df["Clean_Description"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [22]:
df_T = df_train.append(df_val)
df_T.reset_index(drop=True,inplace=True)

In [23]:
corpus = create_corpus_tk(df_T)

In [24]:
num_words = len(corpus)
print(num_words)

72684


In [25]:
corpus[0]

['enjoy',
 'chatting',
 'social',
 'video',
 'chat',
 'face',
 'chat',
 'face',
 'chat',
 'opens',
 'new',
 'doors',
 'video',
 'chatting',
 'people',
 'start',
 'chatting',
 'text',
 'audio',
 'video',
 'options',
 'anytime',
 'anywhere',
 'novel',
 'fun',
 'features',
 'waiting',
 'discover',
 'enjoy',
 'time',
 'face',
 'chat',
 'now',
 'face',
 'chat',
 'chat',
 'people',
 'social',
 'friends',
 'real',
 'time',
 'translation',
 'private',
 'secure',
 'video',
 'chat',
 'quick',
 'call',
 'easy',
 'login',
 'instant',
 'messages',
 'im',
 'beauty',
 'effect',
 'vip',
 'membership',
 'benefits',
 'unlock',
 'text',
 'photos',
 'audio',
 'messages',
 'enjoy',
 'app',
 'without',
 'ads',
 'recommended',
 'users',
 'vip',
 'member',
 'unlock',
 'instant',
 'video',
 'audio',
 'calls',
 'feature',
 'video',
 'audio',
 'chat',
 'users',
 'purchasing',
 'gems',
 'gems',
 'benefits',
 'gems',
 'used',
 'video',
 'audio',
 'call',
 'gems',
 'used',
 'buy',
 'different',
 'gifts',
 'privacy'

##4. Train Validation Split

In [ ]:
# split the data into a training set and a validation set

In [26]:
train_inputs = df_train["Clean_Description"]
validation_inputs = df_val["Clean_Description"]
train_labels = df_train[target_list]
validation_labels = df_val[target_list]

## 5. Tokenization

In [27]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_inputs)

In [28]:
MAX_SEQUENCE_LENGTH = 600

In [29]:
train_sequences = tokenizer.texts_to_sequences(train_inputs)

In [30]:
train_inputs

0        enjoy chatting social video chat face chat fac...
1        running counter calorie counter app way perfec...
2        fake mask neon angel one fake mask neon angel ...
3        live wallpaper app shirokuma days  set app hom...
4        control airsoft gun smartphone tablet android ...
                               ...                        
58142    free messages sms messages instant messaging a...
58143    best online study app competitive exams rasrps...
58144    anime aesthetic wallpapers cool app brings bes...
58145    report corruption report corruption initiative...
58146    speak app designed aid visually impaired other...
Name: Clean_Description, Length: 58147, dtype: object

In [31]:
train_sequences

[[45,
  1383,
  122,
  34,
  171,
  449,
  171,
  449,
  171,
  4165,
  9,
  5770,
  34,
  1383,
  116,
  120,
  1383,
  50,
  231,
  34,
  299,
  516,
  292,
  5729,
  80,
  10,
  722,
  520,
  45,
  12,
  449,
  171,
  207,
  449,
  171,
  171,
  116,
  122,
  44,
  113,
  12,
  360,
  363,
  265,
  34,
  171,
  234,
  84,
  22,
  951,
  529,
  166,
  3134,
  782,
  453,
  3565,
  2164,
  827,
  540,
  50,
  106,
  231,
  166,
  45,
  1,
  102,
  316,
  1388,
  93,
  3565,
  1324,
  540,
  529,
  34,
  231,
  351,
  124,
  34,
  231,
  171,
  93,
  3109,
  5013,
  5013,
  827,
  5013,
  96,
  34,
  231,
  84,
  5013,
  96,
  469,
  52,
  1728,
  176,
  550,
  26370,
  41,
  2087,
  55,
  197,
  648,
  39,
  143,
  6700,
  5322,
  2586,
  55,
  4825,
  16594,
  14442,
  7122,
  11433,
  116,
  1268,
  9135],
 [706,
  1164,
  2416,
  1164,
  1,
  67,
  201,
  319,
  1261,
  764,
  504,
  571,
  447,
  706,
  2416,
  1164,
  2,
  706,
  1,
  55,
  506,
  41,
  420,
  2299,
  319,
  504,

In [32]:
train_padded = pad_sequences(
    train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating="post", padding="post"
)

In [33]:
train_padded

array([[   45,  1383,   122, ...,     0,     0,     0],
       [  706,  1164,  2416, ...,     0,     0,     0],
       [  891,  2345,   617, ...,     0,     0,     0],
       ...,
       [  553,  2136,    13, ...,     0,     0,     0],
       [  762, 13538,   762, ...,     0,     0,     0],
       [  637,     1,   153, ...,     0,     0,     0]], dtype=int32)

In [34]:
validation_sequences = tokenizer.texts_to_sequences(validation_inputs)
validation_padded = pad_sequences(
    validation_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [35]:
validation_padded

array([[ 2316,   209,   425, ...,     0,     0,     0],
       [  479,    21,  1097, ...,     0,     0,     0],
       [   73,    27,  4548, ...,     0,     0,     0],
       ...,
       [  518,    43,    61, ...,     0,     0,     0],
       [    7,  1990, 16433, ...,     0,     0,     0],
       [  440,    67,   401, ...,     0,     0,     0]], dtype=int32)

In [36]:
print(df_train.Clean_Description[0])
print(train_sequences[0])

enjoy chatting social video chat face chat face chat opens new doors video chatting people start chatting text audio video options anytime anywhere novel fun features waiting discover enjoy time face chat now face chat chat people social friends real time translation private secure video chat quick call easy login instant messages im beauty effect vip membership benefits unlock text photos audio messages enjoy app without ads recommended users vip member unlock instant video audio calls feature video audio chat users purchasing gems gems benefits gems used video audio call gems used buy different gifts privacy policy facechat need responsible information provide party please take caution delivering sensitive information sexual pornographic nude behaviors forbidden people rules banned
[45, 1383, 122, 34, 171, 449, 171, 449, 171, 4165, 9, 5770, 34, 1383, 116, 120, 1383, 50, 231, 34, 299, 516, 292, 5729, 80, 10, 722, 520, 45, 12, 449, 171, 207, 449, 171, 171, 116, 122, 44, 113, 12, 360, 3

In [37]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 140253


In [38]:
word_index

{'app': 1,
 'free': 2,
 'use': 3,
 'keyboard': 4,
 'phone': 5,
 'wallpaper': 6,
 'theme': 7,
 'application': 8,
 'new': 9,
 'features': 10,
 'get': 11,
 'time': 12,
 'wallpapers': 13,
 'screen': 14,
 'also': 15,
 'best': 16,
 'like': 17,
 'one': 18,
 'make': 19,
 'download': 20,
 'android': 21,
 'easy': 22,
 'english': 23,
 'us': 24,
 'live': 25,
 'find': 26,
 'share': 27,
 'help': 28,
 'launcher': 29,
 'mobile': 30,
 'learn': 31,
 'love': 32,
 'apps': 33,
 'video': 34,
 'set': 35,
 'device': 36,
 'want': 37,
 'using': 38,
 'please': 39,
 'hd': 40,
 'need': 41,
 'support': 42,
 'home': 43,
 'friends': 44,
 'enjoy': 45,
 'images': 46,
 'game': 47,
 'simple': 48,
 'many': 49,
 'text': 50,
 'themes': 51,
 'different': 52,
 'language': 53,
 'games': 54,
 'information': 55,
 'available': 56,
 'data': 57,
 'day': 58,
 'access': 59,
 'beautiful': 60,
 'play': 61,
 'choose': 62,
 'add': 63,
 'easily': 64,
 'create': 65,
 'lock': 66,
 'way': 67,
 'you': 68,
 'words': 69,
 'save': 70,
 'status':

In [39]:
word_index["reason"]

2469

In [40]:
print(validation_sequences[0])

[2316, 209, 425, 3680, 211, 154, 329, 2, 2316, 209, 425, 3680, 211, 329, 2, 2316, 211, 425, 2316, 209, 425, 3680, 211, 3680, 329, 1536, 48140, 1747, 257, 45, 18, 4538, 853, 48140, 2223, 580, 1606, 80, 39, 20, 2316, 607, 1, 20, 2316, 607, 1, 426, 2316, 211, 329, 5, 2316, 8, 3680, 211, 1, 13497, 209, 425, 2718, 329, 3680, 211, 1155, 13497, 1175, 8349, 2347, 3680, 211, 607, 410, 56, 221, 2718, 3228, 3, 885, 2718, 2316, 211, 27895, 135, 329, 307, 1073, 2316, 211, 329, 4915, 800, 18, 22, 3, 8, 2316, 329, 3680, 2316, 211, 2718, 211, 1125, 514, 13497, 13496, 211, 54378, 211, 1, 201, 3301, 496, 90, 323, 586, 2316, 211, 44, 509, 37, 1084, 733, 3475, 61, 2718, 211, 1, 201, 442, 1486, 124, 2316, 209, 425, 3680, 2316, 211, 329, 35, 209, 453, 607, 35, 104, 2316, 329, 35, 347, 2316, 329, 35, 274, 2316, 329, 65, 2316, 3626, 333, 72, 12, 5, 3658, 1956, 2316, 209, 425, 3680, 2316, 211, 425, 208, 61, 209, 2718, 211, 329, 2, 22, 3, 863, 2, 2828, 8912, 22, 4057, 307, 2316, 329, 63, 329, 835, 201, 1606, 23

In [41]:
word_index["listen"]

413

## 6. Create the Embedding dictionary

In [42]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove6B/glove.6B.300d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [43]:
embedding_dict

Output hidden; open in https://colab.research.google.com to view.

In [44]:
embedding_dim = 300

In [45]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [46]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.73247999, -0.078309  ,  0.018521  , ...,  0.21988   ,
        -0.13121   ,  0.043819  ],
       [-0.30414   , -0.37029999, -0.15881   , ..., -0.37830999,
        -0.42910001,  0.0030023 ],
       ...,
       [-0.15204   , -0.44435   , -0.23480999, ...,  0.35668999,
         0.61373001, -0.14718001],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.50379997, -0.096152  , -0.13287   , ...,  0.49906999,
        -0.53661001,  0.42375001]])

In [47]:
word_index["reason"]

2469

In [48]:
embedding_dict.get("reason")

array([ 1.3197e-01, -1.2591e-01,  4.3864e-02,  3.6321e-02,  9.6646e-02,
       -1.3829e-01,  3.8637e-01,  7.6962e-02, -1.1306e-01, -1.6083e+00,
        2.0062e-02, -5.2665e-02, -1.6597e-01,  1.2171e-01,  2.8945e-01,
       -1.7289e-01,  5.3035e-02, -2.7842e-01,  8.2376e-02, -1.1980e-02,
        3.7228e-02,  2.1867e-01,  1.5267e-01, -8.4361e-02, -3.1292e-01,
       -3.2093e-02,  2.0281e-01, -3.5910e-01,  1.6873e-02, -2.2996e-01,
        2.6044e-02,  3.5910e-01, -3.2431e-01, -5.4194e-01, -9.7742e-01,
        3.9198e-02, -1.7794e-01,  7.4200e-02, -4.1251e-02, -7.8917e-02,
        2.0646e-01, -6.6538e-02,  6.7401e-02,  1.4965e-01,  5.9107e-02,
       -3.7585e-02, -3.4672e-02,  5.5291e-02, -8.5636e-02,  9.1743e-02,
        4.9125e-01,  7.5606e-03, -3.0860e-01,  5.8902e-04, -8.6975e-02,
        3.9904e-01, -1.2695e-01,  2.2471e-01,  2.3658e-01,  3.0489e-01,
       -6.7363e-02,  3.5839e-01,  4.9703e-01,  4.1895e-01, -3.8494e-01,
       -2.6257e-01,  1.6049e-01, -1.0992e-01,  2.7477e-02,  1.49

In [49]:
(embedding_matrix[2363] == embedding_dict.get("reason")).all()

False

In [50]:
print(train_padded.shape)
print(train_labels.shape)

(58147, 600)
(58147, 9)


In [51]:
print(validation_padded.shape)
print(validation_labels.shape)

(14537, 600)
(14537, 9)


## 7. Build CNN Model

In [52]:
num_permissions = 9 #11
drop = 0.2

batch_size = 32
max_train_epochs = 300
validation_split = 6

early_stopping_patience = 16
early_stopping_delta = 0.02  # 2%

max_description_embeddings = 600
embedding_dim = 300  # +1 for flag

#downloaded_embedding_file = data_folder + "/word_embeddings/glove.6B.300d.txt"


conv_filters_num = 1024
conv_filters_sizes = [1, 2, 3]
dense_layers = [5000, 2500]
dropout = 0.2

heatmap_threshold = 0.49

In [53]:
# Function for class weights

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)

    print(class_series)
    print(class_labels)

    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(n_samples)
    print(n_classes)

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))
    #return class_weights

In [54]:
class_series = np.array(train_labels)
class_wt = generate_class_weights(class_series, multi_class=False, one_hot_encoded=True)
print(class_wt)
#class_wt = torch.tensor(class_wt)
#print(class_wt)

58147
9
{0: 0.6950809873886797, 1: 0.7016483251279081, 2: 1.1826428295401388, 3: 1.02308436702736, 4: 0.2267098665793311, 5: 0.7220359608602791, 6: 23.408615136876005, 7: 35.498778998778995, 8: 6.260443583118001}


In [55]:
def model_multiconv_1d(num_permissions):
    #embedding_dim = embedding_dim
    sequence_length = max_description_embeddings

    input_layer = Input(shape=(None,))

    conv_layers = []
    for filter_size in conv_filters_sizes:
        conv_layer_i = Embedding(num_words,
                                 output_dim=embedding_dim,
                                 input_length=sequence_length,
                                 weights=[embedding_matrix],
                                 trainable=False)(input_layer)
        conv_layer_i = Conv1D(filters=conv_filters_num,
                              kernel_size=filter_size,
                              padding='same',
                              activation='relu')(conv_layer_i)
        conv_layer_i = GlobalMaxPooling1D()(conv_layer_i)

        conv_layers.append(conv_layer_i)

    if len(conv_layers) == 1:
        previous_layer = conv_layers[0]
    else:
        concatenated_layer = concatenate(conv_layers, axis=-1)
        previous_layer = concatenated_layer

    for n_neurons in dense_layers:
        previous_layer = Dense(n_neurons, activation='relu')(previous_layer)
        previous_layer = Dropout(dropout)(previous_layer)

    output_layer = Dense(num_permissions, activation='sigmoid')(previous_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [56]:
model = model_multiconv_1d(num_permissions)

In [57]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])
              
#metrics=[metrics.fb_micro, metrics.fb_macro, metrics.precision, metrics.recall])
train_metric = 'val_fb_macro'

In [58]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    42076200    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    42076200    ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    42076200    ['input_1[0][0]']                
                                                                                              

In [59]:
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(train_padded, train_labels, 
          batch_size=batch_size, epochs=10, class_weight=class_wt,
          verbose=1, callbacks=[checkpoint], validation_data=(validation_padded, validation_labels))

Epoch 1/10
1818/1818 [==============================] - 361s 192ms/step - loss: 0.1721 - accuracy: 0.2635 - val_loss: 0.2431 - val_accuracy: 0.2992
Epoch 2/10
1818/1818 [==============================] - 335s 184ms/step - loss: 0.1402 - accuracy: 0.2889 - val_loss: 0.2285 - val_accuracy: 0.2881
Epoch 3/10
1818/1818 [==============================] - 334s 184ms/step - loss: 0.1161 - accuracy: 0.3041 - val_loss: 0.2308 - val_accuracy: 0.2826
Epoch 4/10
1818/1818 [==============================] - 348s 191ms/step - loss: 0.0916 - accuracy: 0.3067 - val_loss: 0.3471 - val_accuracy: 0.3334
Epoch 5/10
1818/1818 [==============================] - 333s 183ms/step - loss: 0.0778 - accuracy: 0.3129 - val_loss: 0.2347 - val_accuracy: 0.3010
Epoch 6/10
1818/1818 [==============================] - 333s 183ms/step - loss: 0.0629 - accuracy: 0.3269 - val_loss: 0.2584 - val_accuracy: 0.3059
Epoch 7/10
1818/1818 [==============================] - 347s 191ms/step - loss: 0.0531 - accuracy: 0.3396 - val_

## 8. Performance on Test Set

In [60]:
test_labels = df_test[target_list]
test_input = df_test["Clean_Description"]

In [61]:
test_sequences = tokenizer.texts_to_sequences(test_input)
test_padded = pad_sequences(
    test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [62]:
print(df_test.Clean_Description[0])
print(test_sequences[0])

poetry lover find best poetry poets urdu problem solved app app contains time best urdu poetry famous urdu poets offline urdu poetry collection best urdu poetry famous urdu poets like mohsin naqvi mirza ghalib saghir siddiqui others also contains romantic urdu poetry sad urdu poetry urdu ghazals offline urdu shayari urdu shayari app contains best ever poetry famous poets times app contains best variety urdu poetry categories romantic poetry sad poetry attitude poetry ghazals inspirational poetry main features offline poetry collection gallery view poetry collection zooming feature save share poetry images app contains offline poetry urdu ghazals urdu poetry poetry famous poets urdu shayari much more
[2888, 1859, 26, 16, 2888, 9848, 372, 683, 2389, 1, 1, 210, 12, 16, 372, 2888, 1008, 372, 9848, 146, 372, 2888, 135, 16, 372, 2888, 1008, 372, 9848, 17, 31576, 35911, 20683, 25306, 43491, 345, 15, 210, 528, 372, 2888, 1567, 372, 2888, 372, 10770, 146, 372, 629, 372, 629, 1, 210, 16, 506, 28

In [63]:
print(test_labels)

      Camera  Location  Microphone  Contacts  Storage  Phone  SMS  Call_Log  \
0          0         0           0         0        1      0    0         0   
1          0         0           0         0        1      0    0         0   
2          0         0           0         0        1      0    0         0   
3          0         0           0         0        0      0    0         0   
4          1         0           0         0        0      1    0         0   
...      ...       ...         ...       ...      ...    ...  ...       ...   
4619       0         1           0         0        0      0    0         0   
4620       0         0           0         0        0      0    0         0   
4621       0         0           0         0        1      0    0         0   
4622       0         1           1         0        1      0    0         0   
4623       0         0           0         0        0      0    0         0   

      Calendar  
0            0  
1            0  


In [64]:
print(test_padded.shape)
print(test_labels.shape)

(4624, 600)
(4624, 9)


In [65]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_padded)
print("predictions shape:", predictions.shape)

predictions shape: (4624, 9)


In [66]:
predictions[0:10]

array([[5.19189809e-04, 3.47143435e-03, 4.49706249e-07, 8.36062242e-08,
        5.46902299e-01, 1.30500106e-04, 5.19412569e-10, 1.90678975e-13,
        5.56441648e-08],
       [9.15107421e-07, 2.11107889e-07, 1.48602084e-07, 1.20221364e-06,
        8.42217414e-05, 1.36220669e-05, 6.62844641e-08, 1.02077227e-11,
        7.44791464e-07],
       [5.89677572e-01, 6.02620239e-05, 7.81145811e-01, 8.30957443e-02,
        9.64024127e-01, 3.36451381e-01, 3.13253731e-05, 1.86495581e-05,
        4.38576238e-03],
       [1.39754900e-06, 1.73076798e-04, 2.19481353e-06, 7.04500675e-02,
        6.39185369e-01, 2.44896542e-02, 6.20430455e-06, 2.58685361e-07,
        2.80702458e-04],
       [1.67306315e-03, 1.56730579e-07, 1.59214810e-03, 3.16011719e-05,
        9.86729050e-04, 1.64806917e-02, 2.06382947e-06, 6.03060926e-08,
        5.14504572e-05],
       [5.33856928e-01, 3.75799194e-04, 5.59967943e-03, 1.58252850e-01,
        9.90025997e-01, 3.87178957e-02, 2.03527277e-04, 3.97563599e-06,
        1.2

In [67]:
true_labels = test_labels.to_numpy()
true_labels[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0]])

In [68]:
np.save("predictions.npy", predictions)
loaded_predictions = np.load("predictions.npy")
print(loaded_predictions)

[[5.1918981e-04 3.4714343e-03 4.4970625e-07 ... 5.1941257e-10
  1.9067897e-13 5.5644165e-08]
 [9.1510742e-07 2.1110789e-07 1.4860208e-07 ... 6.6284464e-08
  1.0207723e-11 7.4479146e-07]
 [5.8967757e-01 6.0262024e-05 7.8114581e-01 ... 3.1325373e-05
  1.8649558e-05 4.3857624e-03]
 ...
 [9.9014562e-01 4.3365739e-02 2.8966505e-02 ... 9.5359376e-03
  5.3584820e-04 1.4190686e-02]
 [2.2045444e-05 9.9987185e-01 9.9995911e-01 ... 4.5278221e-14
  1.0996109e-13 1.3032526e-04]
 [2.5000065e-03 9.2012084e-07 1.5552067e-05 ... 4.4768865e-07
  9.3425900e-10 1.6458526e-06]]


In [69]:
np.save("true_labels.npy", true_labels)
loaded_true_labels = np.load("true_labels.npy")
print(loaded_true_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 9. Threshold Calculation

In [70]:
# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [71]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [72]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.5898 with F-Score: 0.676
permission  1
Length of sequence: 10000
Best Threshold: 0.0567 with F-Score: 0.5682
permission  2
Length of sequence: 10000
Best Threshold: 0.0632 with F-Score: 0.577
permission  3
Length of sequence: 10000
Best Threshold: 0.0613 with F-Score: 0.5347
permission  4
Length of sequence: 10000
Best Threshold: 0.1777 with F-Score: 0.7935
permission  5
Length of sequence: 10000
Best Threshold: 0.0934 with F-Score: 0.5284
permission  6
Length of sequence: 10000
Best Threshold: 0.1446 with F-Score: 0.1818
permission  7
Length of sequence: 10000
Best Threshold: 0.1413 with F-Score: 0.4444
permission  8
Length of sequence: 10000
Best Threshold: 0.1105 with F-Score: 0.375
-------------------------------------
optimal threshold tuning for f-score
[0.5898, 0.0567, 0.0632, 0.0613, 0.1777, 0.0934, 0.1446, 0.1413, 0.1105]


In [73]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.08470000326633453 with G-Mean: 0.8443
FPR: 0.149, TPR: 0.8376
permission  1
Best Threshold: 0.003100000089034438 with G-Mean: 0.778
FPR: 0.1776, TPR: 0.736
permission  2
Best Threshold: 0.0017999999690800905 with G-Mean: 0.8235
FPR: 0.1944, TPR: 0.8417
permission  3
Best Threshold: 0.0038999998942017555 with G-Mean: 0.8045
FPR: 0.1848, TPR: 0.794
permission  4
Best Threshold: 0.34709998965263367 with G-Mean: 0.7837
FPR: 0.2214, TPR: 0.7889
permission  5
Best Threshold: 0.012199999764561653 with G-Mean: 0.7651
FPR: 0.2115, TPR: 0.7423
permission  6
Best Threshold: 0.0008999999845400453 with G-Mean: 0.8715
FPR: 0.0718, TPR: 0.8182
permission  7
Best Threshold: 0.0 with G-Mean: 0.9541
FPR: 0.0896, TPR: 1.0
permission  8
Best Threshold: 0.0010999999940395355 with G-Mean: 0.8325
FPR: 0.2104, TPR: 0.8778
-------------------------------------
ROC curve with G-mean threshold tuning
[0.0847, 0.0031, 0.0018, 0.0039, 0.3471, 0.0122, 0.0009, 0.0, 0.0011]


## 10. Performance Score

In [ ]:
#Fscore micro for different thresholds-

In [74]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [75]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f_at_1(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

   
    acc[0] = f1_score(labels[:, 0], preds_th[:, 0])
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1])
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2])
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3])
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4])
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5])
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6])
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7])
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8])

   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [76]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("F1_CV3_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.6760
  Location  : 0.5682
  Microphone: 0.5770
  Contacts  : 0.5347
  Storage   : 0.7935
  Phone     : 0.5284
  SMS       : 0.1818
  Call_Log  : 0.4444
  Calendar  : 0.3750

  Average F1 score: 0.5199


In [ ]:
#Fscore micro for different thresholds-

In [77]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [78]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f1micro_accuracy(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

    acc[0] = f1_score(labels[:, 0], preds_th[:, 0], average='micro')
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1], average='micro')
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2], average='micro')
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3], average='micro')
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4], average='micro')
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5], average='micro')
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6], average='micro')
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7], average='micro')
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8], average='micro')
    
   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [79]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("F1Mic_CV3_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8895
  Location  : 0.8679
  Microphone: 0.9109
  Contacts  : 0.8856
  Storage   : 0.7755
  Phone     : 0.8599
  SMS       : 0.9961
  Call_Log  : 0.9989
  Calendar  : 0.9762

  Average F1 (micro) score: 0.9067


In [ ]:
#roc-auc score for different thresholds-

In [80]:
import numpy as np
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [81]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import roc_auc_score

def roc_auc(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_roc[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_roc[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_roc[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_roc[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_roc[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_roc[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_roc[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_roc[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_roc[8]


    acc[0] = roc_auc_score(labels[:, 0], preds_th[:, 0])
    acc[1] = roc_auc_score(labels[:, 1], preds_th[:, 1])
    acc[2] = roc_auc_score(labels[:, 2], preds_th[:, 2])
    acc[3] = roc_auc_score(labels[:, 3], preds_th[:, 3])
    acc[4] = roc_auc_score(labels[:, 4], preds_th[:, 4])
    acc[5] = roc_auc_score(labels[:, 5], preds_th[:, 5])
    acc[6] = roc_auc_score(labels[:, 6], preds_th[:, 6])
    acc[7] = roc_auc_score(labels[:, 7], preds_th[:, 7])
    acc[8] = roc_auc_score(labels[:, 8], preds_th[:, 8])
   

    #print(acc)
    return acc

In [82]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ROC_CV3_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.8436
  Location  : 0.7785
  Microphone: 0.8230
  Contacts  : 0.8043
  Storage   : 0.7835
  Phone     : 0.7654
  SMS       : 0.8727
  Call_Log  : 0.5000
  Calendar  : 0.8337

  Average ROC_AUC score: 0.7783


# ACNET performance

#### a) Data Preparation

In [83]:
## drive path
acnet_path = "/content/drive/MyDrive/MetadataCSV/acnet_dataset_preprocess.csv"

In [84]:
df_acnet = pd.read_csv(acnet_path) 
print(df_acnet.shape)

(1417, 14)


In [85]:
df_acnet = df_acnet.dropna(subset=['Clean_Description'])

In [86]:
df_acnet.head()

,app_id,description,Storage,Contacts,Location,Camera,Microphone,SMS,Call_Log,Phone,Calendar,Settings,Tasks,Clean_Description
0,0,ROOT is REQUIRED for automatic synchronization...,1,0,1,0,0,0,0,0,0,0,0,root is required for automatic synchronization...
1,1,This app delivers short scriptures containing ...,0,0,0,0,0,1,0,0,0,0,0,this app delivers short scriptures containing ...
2,2,This game is surprisingly simple and very addi...,0,0,0,0,0,0,0,0,0,0,0,this game is surprisingly simple and very addi...
3,3,It is an online RPG game based on LBS location...,0,0,1,0,0,0,0,0,0,0,0,it is an online rpg game based on lbs location...
4,4,Christmas is in the air. Get yourself in the h...,0,1,0,0,0,0,0,0,0,1,1,christmas is in the air. get yourself in the h...


In [87]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(remove_stopwords)

In [88]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(lambda x: remove_punct(x))

In [89]:
acnet_labels = df_acnet[target_list]
acnet_input = df_acnet["Clean_Description"]

In [90]:
acnet_sequences = tokenizer.texts_to_sequences(acnet_input)
acnet_padded = pad_sequences(
    acnet_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [91]:
print(acnet_padded.shape)
print(acnet_labels.shape)

(1414, 600)
(1414, 9)


#### b) Get Predictions

In [92]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(acnet_padded)
print("predictions shape:", predictions.shape)

predictions shape: (1414, 9)


In [93]:
acnet_labels = acnet_labels.to_numpy()

In [94]:
np.save("acnet_predictions.npy", predictions)
loaded_predictions = np.load("acnet_predictions.npy")

np.save("acnet_labels.npy", acnet_labels)
loaded_true_labels = np.load("acnet_labels.npy")

#### c) Threshold Calculation

In [95]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [96]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.7661 with F-Score: 0.464
permission  1
Length of sequence: 10000
Best Threshold: 0.039 with F-Score: 0.5174
permission  2
Length of sequence: 10000
Best Threshold: 0.6447 with F-Score: 0.5495
permission  3
Length of sequence: 10000
Best Threshold: 0.0226 with F-Score: 0.6266
permission  4
Length of sequence: 10000
Best Threshold: 0.0004 with F-Score: 0.5827
permission  5
Length of sequence: 10000
Best Threshold: 0.8003 with F-Score: 0.3936
permission  6
Length of sequence: 10000
Best Threshold: 0.001 with F-Score: 0.5674
permission  7
Length of sequence: 10000
Best Threshold: 0.0017 with F-Score: 0.2995
permission  8
Length of sequence: 10000
Best Threshold: 0.3194 with F-Score: 0.5195
-------------------------------------
optimal threshold tuning for f-score
[0.7661, 0.039, 0.6447, 0.0226, 0.0004, 0.8003, 0.001, 0.0017, 0.3194]


In [97]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.2955999970436096 with G-Mean: 0.6846
FPR: 0.2388, TPR: 0.6157
permission  1
Best Threshold: 0.0027000000700354576 with G-Mean: 0.6991
FPR: 0.3153, TPR: 0.7138
permission  2
Best Threshold: 0.015300000086426735 with G-Mean: 0.7908
FPR: 0.2156, TPR: 0.7973
permission  3
Best Threshold: 0.010099999606609344 with G-Mean: 0.7307
FPR: 0.2697, TPR: 0.731
permission  4
Best Threshold: 0.4339999854564667 with G-Mean: 0.5559
FPR: 0.4755, TPR: 0.5893
permission  5
Best Threshold: 0.03779999911785126 with G-Mean: 0.7362
FPR: 0.3285, TPR: 0.807
permission  6
Best Threshold: 9.999999747378752e-05 with G-Mean: 0.79
FPR: 0.2396, TPR: 0.8208
permission  7
Best Threshold: 0.0 with G-Mean: 0.7483
FPR: 0.2534, TPR: 0.75
permission  8
Best Threshold: 0.00570000009611249 with G-Mean: 0.8298
FPR: 0.2382, TPR: 0.9038
-------------------------------------
ROC curve with G-mean threshold tuning
[0.2956, 0.0027, 0.0153, 0.0101, 0.434, 0.0378, 1e-04, 0.0, 0.0057]


#### d) Accuracy Score

In [ ]:
#Fscore micro for different thresholds-

In [98]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [99]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("ACNET_F1_CV3_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.4640
  Location  : 0.5174
  Microphone: 0.5495
  Contacts  : 0.6266
  Storage   : 0.5827
  Phone     : 0.3936
  SMS       : 0.5674
  Call_Log  : 0.2995
  Calendar  : 0.5195

  Average F1 score: 0.5022


In [ ]:
#Fscore micro for different thresholds-

In [100]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [101]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("ACNET_F1Mic_CV3_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8105
  Location  : 0.7744
  Microphone: 0.9130
  Contacts  : 0.7539
  Storage   : 0.4236
  Phone     : 0.9194
  SMS       : 0.8479
  Call_Log  : 0.8975
  Calendar  : 0.9477

  Average F1 (micro) score: 0.8098


In [ ]:
#roc-auc score for different thresholds-

In [102]:
import numpy as np
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [103]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ACNET_ROC_CV3_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.6885
  Location  : 0.6993
  Microphone: 0.7878
  Contacts  : 0.7307
  Storage   : 0.5569
  Phone     : 0.7393
  SMS       : 0.7787
  Call_Log  : 0.5000
  Calendar  : 0.8325

  Average ROC_AUC score: 0.7015
